In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- 1. CONFIGURATION ---
EMBEDDING_DIM = 64     
MARGIN = 0.5           
EPOCHS = 30            
BATCH_SIZE = 4         
LEARNING_RATE = 1e-4   
TEXT_MODEL_NAME = "distilbert-base-uncased" 

# --- HHAR CONFIGURATION ---
SENSOR_FEATURES_COUNT = 6 # Acc_x, Acc_y, Acc_z, Gyro_X, Gyro_Y, Gyro_Z
TEXT_COLUMN_NAME = 'Semantic_Interpretation'
SENSOR_COLUMNS = ['Acc_x', 'Acc_y', 'Acc_z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z']
SENSOR_MODEL_PATH = 'sensor_encoder_hhar_6col.pth'
TEXT_MODEL_PATH = 'text_encoder_hhar_6col.pth'
DATA_FILE = "HHAR_semantic_interpretation.csv" 

# --- 2. DATASET CLASS ---

class SensorTextDataset(Dataset):
    def __init__(self, sensor_data, text_data, labels):
        self.sensor_data = torch.tensor(sensor_data, dtype=torch.float32)
        self.text_data = text_data
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sensor_data[idx], self.text_data[idx], self.labels[idx]

# --- 3. DUAL-ENCODER ARCHITECTURE ---

class SensorEncoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        return self.encoder(x)

class TextEncoder(nn.Module):
    def __init__(self, model_name, output_dim):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.projection = nn.Linear(self.model.config.hidden_size, output_dim)
    def forward(self, texts):
        if isinstance(texts, torch.Tensor):
             texts = texts.tolist()
             
        encoded_input = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
        
        device = self.projection.weight.device
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        
        output = self.model(**encoded_input)
        mean_pooled = torch.mean(output.last_hidden_state, dim=1)
        return self.projection(mean_pooled)

# --- 4. CONTRASTIVE LOSS FUNCTION (Cosine Similarity) ---

class ContrastiveSimilarityLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(ContrastiveSimilarityLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        similarity = F.cosine_similarity(output1, output2).unsqueeze(1)
        
        device = output1.device
        zero_tensor = torch.tensor(0.0).to(device)
        margin_tensor = torch.tensor(self.margin).to(device)

        loss_positive = label * (1 - similarity)
        loss_negative = (1 - label) * torch.max(zero_tensor, similarity - margin_tensor)
        
        loss = torch.mean(loss_positive + loss_negative)
        return loss

# --- 5. TRAINING FUNCTION ---

def train_contrastive_model():
    print("--- Starting Training (Cosine Similarity Contrastive Loss) ---")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    df = pd.read_csv(DATA_FILE)
    
    sensor_cols = SENSOR_COLUMNS 

    if set(sensor_cols).issubset(df.columns) is False:
        raise ValueError(f"Required sensor columns {sensor_cols} not found in {DATA_FILE}")

    if len(sensor_cols) != SENSOR_FEATURES_COUNT:
        raise ValueError(f"Feature count mismatch! Expected {SENSOR_FEATURES_COUNT} features, but detected {len(sensor_cols)} in the data file: {DATA_FILE}")

    sensor_data = df[sensor_cols].values
    text_data = df[TEXT_COLUMN_NAME].tolist()

    # Create POSITIVE and NEGATIVE training samples
    pos_sensor, pos_text, pos_labels = sensor_data, text_data, np.ones(len(df))
    neg_sensor = sensor_data
    neg_text = np.roll(text_data, 1).tolist()
    neg_labels = np.zeros(len(df))

    all_sensor = np.concatenate([pos_sensor, neg_sensor])
    all_text = pos_text + neg_text
    all_labels = np.concatenate([pos_labels, neg_labels])

    # Split into training and test sets
    X_train_s, X_test_s, X_train_t, X_test_t, y_train, y_test = train_test_split(
        all_sensor, all_text, all_labels, test_size=0.3, random_state=42
    )

    sensor_encoder = SensorEncoder(SENSOR_FEATURES_COUNT, EMBEDDING_DIM).to(device)
    text_encoder = TextEncoder(TEXT_MODEL_NAME, EMBEDDING_DIM).to(device)
    criterion = ContrastiveSimilarityLoss(margin=MARGIN)
    
    params = list(sensor_encoder.parameters()) + list(text_encoder.parameters())
    optimizer = optim.Adam(params, lr=LEARNING_RATE)

    train_dataset = SensorTextDataset(X_train_s, X_train_t, y_train)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    print(f"Total training pairs: {len(train_dataset)}")

    for epoch in range(EPOCHS):
        total_loss = 0
        sensor_encoder.train()
        text_encoder.train()
        
        for sensor_batch, text_batch, label_batch in train_loader:
            optimizer.zero_grad()
            
            sensor_batch = sensor_batch.to(device)
            label_batch = label_batch.to(device)
            
            sensor_embedding = sensor_encoder(sensor_batch)
            text_embedding = text_encoder(text_batch)
            
            loss = criterion(sensor_embedding, text_embedding, label_batch.unsqueeze(1))
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS}, Average Loss: {avg_loss:.4f}")

    print("\nTraining complete.")
    
    torch.save(sensor_encoder.state_dict(), SENSOR_MODEL_PATH)
    torch.save(text_encoder.state_dict(), TEXT_MODEL_PATH)
    print(f"Models saved: {SENSOR_MODEL_PATH} and {TEXT_MODEL_PATH}")

    return X_test_s, X_test_t, y_test

# --- 6. EVALUATION FUNCTION ---

def evaluate_contrastive_model(X_test_s, X_test_t, y_test):
    print("\n--- Starting Evaluation on Test Data ---")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    sensor_encoder = SensorEncoder(SENSOR_FEATURES_COUNT, EMBEDDING_DIM).to(device)
    text_encoder = TextEncoder(TEXT_MODEL_NAME, EMBEDDING_DIM).to(device)

    try:
        sensor_encoder.load_state_dict(torch.load(SENSOR_MODEL_PATH, map_location=device))
        text_encoder.load_state_dict(torch.load(TEXT_MODEL_PATH, map_location=device))
    except FileNotFoundError:
        print(f"Error: Model files not found. Please ensure {SENSOR_MODEL_PATH} and {TEXT_MODEL_PATH} exist.")
        return

    sensor_encoder.eval()
    text_encoder.eval()
    
    test_dataset = SensorTextDataset(X_test_s, X_test_t, y_test)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    similarities = []
    true_labels = []

    with torch.no_grad():
        for sensor_batch, text_batch, label_batch in test_loader:
            sensor_batch = sensor_batch.to(device)
            
            sensor_embedding = sensor_encoder(sensor_batch)
            text_embedding = text_encoder(text_batch)
            
            sim = F.cosine_similarity(sensor_embedding, text_embedding)
            similarities.extend(sim.cpu().numpy())
            true_labels.extend(label_batch.cpu().numpy())

    similarities = np.array(similarities)
    true_labels = np.array(true_labels)
    
    pos_similarities = similarities[true_labels == 1.0]
    neg_similarities = similarities[true_labels == 0.0]

    print(f"Test Set Size: {len(true_labels)}")
    print(f"Mean Similarity (Positive Pairs): {np.mean(pos_similarities):.4f}")
    print(f"Mean Similarity (Negative Pairs): {np.mean(neg_similarities):.4f}")
    
    predictions = (similarities > MARGIN).astype(int)
    
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Matching Accuracy (Similarity > MARGIN={MARGIN}): {accuracy * 100:.2f}%")


if __name__ == "__main__":
    try:
        X_test_s, X_test_t, y_test = train_contrastive_model()
        evaluate_contrastive_model(X_test_s, X_test_t, y_test)
    except Exception as e:
        # Final reminder for the user to address the environment error locally.
        print("\n----------------------------------------------------------------------")
        print("🛑 **EXECUTION REMINDER** 🛑")
        print(f"An error occurred: {e}")
        print("\n**ACTION REQUIRED:** This persistent error is due to a misconfigured PyTorch installation in your current environment. Please follow the steps above (Clean Up, Create New Env, Reinstall Libraries) to fix your local setup, then run this code again.")
        print("----------------------------------------------------------------------")